In [ ]:

!pip3 install sentence-transformers

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.tag import pos_tag
import re

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


from sentence_transformers import SentenceTransformer
import gensim 
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     /home/terificknight/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/terificknight/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/terificknight/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/terificknight/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-11-12 09:31:10.810529: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 09:31:10.810622: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU

In [ ]:
## Reading data file and converting them into csv files.

#Reading Files.
file1 = open('data2/train.dat', 'r')
Lines = file1.readlines()

file2 = open('data2/test.dat', 'r')
Lines2 = file2.readlines()

#storing data in a list
train_data=[]

for i in Lines:
    x=int(i[0:2])
    train_data.append([x,i[3:-1].replace("<br />","")])


test_text=[]
for i in Lines2:
    test_text.append(i[:-1])
    #print(i)

# Converting list to dataframe and saving into a csv file.
train_data=pd.DataFrame(train_data,columns=["labels","Reviews"],dtype=float)
train_data.to_csv("data2/train_data.csv",index=False)
test_data=pd.DataFrame(test_text,columns=["Reviews"])
test_data.to_csv("data2/test_data.csv",index=False)

del train_data
del test_data
del file1
del file2
del Lines
del Lines2

/home/terificknight/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
train_data=pd.read_csv("data2/train_data.csv")
test_data=pd.read_csv("data2/test_data.csv")

In [ ]:
def mean(z): # used for BERT (word version) and Word2Vec
    return sum(itertools.chain(z))/len(z)

In [ ]:
# def NormalizeWithPOS(text):
#     # Lemmatization & Stemming according to POS tagging

#     word_list = word_tokenize(text)
#     rev = []
#     lemmatizer = WordNetLemmatizer() 
#     stemmer = PorterStemmer() 
#     for word, tag in pos_tag(word_list):
#         if tag.startswith('J'):
#             w = lemmatizer.lemmatize(word, pos='a')
#         elif tag.startswith('V'):
#             w = lemmatizer.lemmatize(word, pos='v')
#         elif tag.startswith('N'):
#             w = lemmatizer.lemmatize(word, pos='n')
#         elif tag.startswith('R'):
#             w = lemmatizer.lemmatize(word, pos='r')
#         else:
#             w = word
#         w = stemmer.stem(w)
#         rev.append(w)
#     review = ' '.join(rev)
#     return review

In [ ]:
def cleanText(text):
    
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'re", " are", text)

    
    # Replace punctuations with space
    # save ! ? . for end of the sentence detection [,/():;']
    filters='"#$%&*+<=>@[\\]^_`{|}~\t\n'
    text = re.sub(r'\!+', '!', text)
    text = re.sub(r'\?+', '?', text)

    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    
    text = re.sub(r'\( *\)', ' ', text)

    # Replace multiple space with one space
    text = re.sub(' +', ' ', text)
    
    text = ''.join(text)

    return text


In [ ]:
def embeddToBERT(text):
    sentences = re.split('!|\?|\.',text)
    sentences = list(filter(None, sentences)) 
    
    ## encoding the sentence
    result = bert_transformers.encode(sentences)
    #sys.stdout.write('\r'+"in")
    feature = [mean(x) for x in zip(*result)]
  
    return np.asarray(feature).reshape((768,1))

## Cleaning Text

In [ ]:
embedding = 'BERT'
# for Word2Vec with stop words
train_data['clean_text'] = train_data['Reviews'].apply(cleanText)
test_data['clean_text'] = test_data['Reviews'].apply(cleanText)

In [ ]:
train_data.head()

,labels,Reviews,clean_text
0,-1.0,Although a film with Bruce Willis is always wo...,Although a film with Bruce Willis is always wo...
1,-1.0,This movie was slower then Molasses in January...,This movie was slower then Molasses in January...
2,-1.0,Interesting film about an actual event that to...,Interesting film about an actual event that to...
3,-1.0,It's painfully obvious that the people who mad...,It's painfully obvious that the people who mad...
4,1.0,This movie really is a mixed bag. On the one h...,This movie really is a mixed bag. On the one h...


## Split data into Traning and Validation dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data['clean_text'], train_data['labels'], test_size=0.33, random_state=321)
X_train.head()

20227    Please...if anybody gets the chance to read th...
9945     This film is absolutely horrific. One of the w...
12379    The Muppet movie is an instant classic. I reme...
3639     Ray Liotta and Tom Hulce shine in this sterlin...
14592    Finally, Timon and Pumbaa in their own film......
Name: clean_text, dtype: object

## Bert Transformer for Converting Text to Vectors

In [ ]:
bert_transformers = SentenceTransformer('bert-base-nli-mean-tokens')

## Converting Training dataset Text to vectors using Bert transformer

In [ ]:
import sys
#bert_versio= 'SENTENCE'
c=0
bert_sentence_training_features=[]
for i in X_train:
    sys.stdout.write('\r'+str(c))
    bert_sentence_training_features.append(embeddToBERT(i))
    c=c+1

16749

In [ ]:
feature = [x.T for x in bert_sentence_training_features]
bert_sentence_training_features = np.asarray(feature).reshape(len(X_train),768)

print(bert_sentence_training_features.shape)


(16750, 768)


In [ ]:
## saving into csv
#np.savetxt('data/embedding_data_final3.csv', bert_sentence_training_features, delimiter=',')

## Converting Validation dataset Text to vectors using Bert transformer

In [ ]:
# bert_sentence_test_features = X_test.apply(embeddToBERT)
#bert_version = 'SENTENCE'
c=0
bert_sentence_test_features=[]
for i in X_test:
    sys.stdout.write('\r'+str(c))
    bert_sentence_test_features.append(embeddToBERT(i))
    c=c+1

8249

In [ ]:
feature = [x.T for x in bert_sentence_test_features]
bert_sentence_test_features = np.asarray(feature).reshape(len(X_test),768)
print(bert_sentence_test_features.shape)

(8250, 768)


## Initialize Svm model with Linear Kernel

In [ ]:
model = SVC(kernel ='linear', C = 1)

## Training the Model

In [ ]:
model.fit(bert_sentence_training_features, y_train)

SVC(C=1, kernel='linear')

## Evaluation of model on validation dataset

In [ ]:
# Evaluation
y_pred_bert_words_svm = model.predict(bert_sentence_test_features)

#y_prob_bert_words_svm = model.decision_function(bert_word_test_features)

print(classification_report(y_test,y_pred_bert_words_svm))

              precision    recall  f1-score   support

        -1.0       0.90      0.91      0.90      4124
         1.0       0.91      0.90      0.90      4126

    accuracy                           0.90      8250
   macro avg       0.90      0.90      0.90      8250
weighted avg       0.90      0.90      0.90      8250



## Converting Test dataset to vectors using Bert Transformer

In [ ]:
#bert_version = 'SENTENCE'
c=0
bert_sentence_training_features=[]
for i in test_data['clean_text']:
    sys.stdout.write('\r'+str(c))
    bert_sentence_training_features.append(embeddToBERT(i))
    c=c+1

24999

In [ ]:
feature = [x.T for x in bert_sentence_training_features]
bert_sentence_training_features = np.asarray(feature).reshape(len(test_data['clean_text']),768)

print(bert_sentence_training_features.shape)

(25000, 768)


## Labelling test data using the model trained above

In [ ]:
y_pred=model.predict(bert_sentence_training_features)
pred=pd.DataFrame(y_pred,dtype=int)
pred.to_csv("data2/final_3_prediction_svc.dat",index=None,header=None)